### Read multiple CSV files at once

In [3]:
from pathlib import Path

In [2]:
Path.cwd()

PosixPath('/home/anchu/Documents/learning/learn-python/cookbook-polars-for-python')

In [3]:
Path.cwd().rglob('data/On_Time*.csv')

<generator object Path.rglob at 0x7f1d4c12ceb0>

In [1]:
def convert_bytes(size):
    """ Convert bytes to KB, or MB or GB"""
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024.0:
            return "%3.1f %s" % (size, x)
        size /= 1024.0

In [6]:
for p in Path.cwd().rglob('data/On_Time*.csv'):
    print(p.name, ":", convert_bytes(p.stat().st_size))

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_5.csv : 249.1 MB
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_1.csv : 229.9 MB
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_8.csv : 253.9 MB
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_6.csv : 248.4 MB
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_7.csv : 256.5 MB
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_3.csv : 243.3 MB
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_2.csv : 212.4 MB
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_12.csv : 239.6 MB
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_4.csv : 239.5 MB
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_9.csv : 239.5 MB
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_10.csv : 246.6 MB
On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2022_11.csv : 235.7 MB


In [5]:
import polars as pl

In [6]:
lazy_df = (
    pl.scan_csv('./data/On_Time*.csv')
    .group_by(['Year', 'Month'])
    .agg(
        pl.len().alias('Rows count')
    )
    .sort('Month')
    .collect()
)

PARTITIONED DS


In [7]:
lazy_df

Year,Month,Rows count
i64,i64,u32
2022,1,537902
2022,2,495713
2022,3,564853
2022,4,556502
2022,5,578819
2022,6,577283
2022,7,594957
2022,8,589810
2022,9,557494


In [9]:
import polars as pl

In [11]:
queries = []
for p in Path.cwd().rglob('data/On_Time*.csv'):
    q = (
        pl.scan_csv(p)
        .group_by(['Year', 'Month'])
        .agg(
            pl.len().alias('Rows Count')
        )
    )
    queries.append(q)

dfs = pl.collect_all(queries)
dfs[:3]

[shape: (1, 3)
 ┌──────┬───────┬────────────┐
 │ Year ┆ Month ┆ Rows Count │
 │ ---  ┆ ---   ┆ ---        │
 │ i64  ┆ i64   ┆ u32        │
 ╞══════╪═══════╪════════════╡
 │ 2022 ┆ 5     ┆ 578819     │
 └──────┴───────┴────────────┘,
 shape: (1, 3)
 ┌──────┬───────┬────────────┐
 │ Year ┆ Month ┆ Rows Count │
 │ ---  ┆ ---   ┆ ---        │
 │ i64  ┆ i64   ┆ u32        │
 ╞══════╪═══════╪════════════╡
 │ 2022 ┆ 1     ┆ 537902     │
 └──────┴───────┴────────────┘,
 shape: (1, 3)
 ┌──────┬───────┬────────────┐
 │ Year ┆ Month ┆ Rows Count │
 │ ---  ┆ ---   ┆ ---        │
 │ i64  ┆ i64   ┆ u32        │
 ╞══════╪═══════╪════════════╡
 │ 2022 ┆ 8     ┆ 589810     │
 └──────┴───────┴────────────┘]